In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.4 MB 6.9 MB/s 


In [3]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=77912055be8d30e0377aa07ab5521230a648fa8797c9526c126b314551b5a87b
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from pandasql import sqldf
from sklearn.preprocessing import StandardScaler
import os
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings(action='ignore')
from matplotlib import font_manager, rc
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# font_path = "C:\\Windows\\Fonts\\\x7f\x7f\x7f\x7fBOLD.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
from sklearn.manifold import TSNE
import re
import talib as tb

In [5]:
def weather(주소,name = 'train'):
    t=0
    if name =='test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):
        globals()[f'{name}_weather_'+str(i)]=pd.DataFrame()

        for k in range(0,3):
            try:
                weather = pd.read_csv(주소+f'/weather_{i}_{t}.csv')
                if len(weather)==0:
                    weather = pd.read_csv(주소+f'/weather_0_0.csv')
                    weather=weather.set_index('datadate')
                    weather[weather.columns] = 0
                    weather=weather.reset_index()
                weather = weather.set_index('datadate')
                weather['습도(%)']=weather['습도(%)'].apply(lambda x : np.nan if x==' ' else x)
                weather['습도(%)']=weather['습도(%)'].astype('float')
                weather.columns = weather.columns+f'_{k%3}'
                globals()[f'{name}_weather_'+str(i)]=pd.merge(globals()[f'{name}_weather_'+str(i)],weather,how='right',left_index=True, right_index=True)    


            except:
                break
            t+=1



        globals()[f'{name}_weather_'+str(i)]=globals()[f'{name}_weather_'+str(i)].reset_index()
        cols = globals()[f'{name}_weather_'+str(i)].columns
        globals()[f'{name}_weather_'+str(i)]['월일']=globals()[f'{name}_weather_'+str(i)]['datadate']%10000
       

        
        if name == 'train':
            qwe =  globals()[f'{name}_weather_'+str(i)].groupby('월일').mean().reset_index()
            qwe.columns = qwe.columns+'_평균'
            globals()[f'train_weather_평균_'+str(i)]= qwe



            


        globals()[f'{name}_weather_'+str(i)] = pd.merge(globals()[f'{name}_weather_'+str(i)],globals()[f'train_weather_평균_'+str(i)],how='left',left_on = '월일',right_on='월일_평균')
        for c in cols:
            globals()[f'{name}_weather_'+str(i)][c]=globals()[f'{name}_weather_'+str(i)].apply(lambda x: x[c] if np.isnan(x[c]) == False else (0 if np.isnan(x[c+'_평균']) else x[c+'_평균']),axis=1)
                
    
        
        
        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_weather_'+str(i)] = globals()[f'{name}_weather_'+str(i)][cols]

        else:
            globals()[f'{name}_weather_'+str(i)] = globals()[f'{name}_weather_'+str(i)][cols]

        

In [6]:
def add_dosomae(주소,name='train', option=1):
        if name == 'test':
            test_dir = 주소.split('/')[-2][-1]
        data_list = glob(주소+'*') # train raw 데이터 넣을지 / test raw 데이터 넣을지 경로
        domae = []
        somae = []

        for i in data_list:
            if 'domae' in i:
                domae.append(i)
            if 'somae' in i:
                somae.append(i)
        
        sum_df = pd.DataFrame()
                
        if option == 1:
            df = domae
            text = 'domae'
        else:
            df = somae
            text = 'somae'


        for i in tqdm(df):
            test = pd.read_csv(i)
            
            k=i.split('/')[-1].split("_")[1].split(".")[0]
            test.fillna(0,inplace=True) # 널값 0으로 채워주고
            if len(test)==0:
                test = pd.read_csv(주소+f'/domae_0.csv')
                test=test.set_index('datadate')
                test[test.columns] = 0
                test=test.reset_index()
                

            
            test['조사단위(kg)'] = test['조사단위(kg)'].replace('g$|kg$|개|포기|접', '',regex=True)
            test=test.astype({'조사단위(kg)':'float'})
            test.loc[test['조사단위(kg)'] >= 100, '단위당가격'] = test['가격(원)']* test['조사단위(kg)'] /1000
            test.loc[test['조사단위(kg)'] < 100, '단위당가격'] = test['가격(원)']/ test['조사단위(kg)']
            
            sep = test.loc[(test['등급명'] == '상품') | (test['등급명'] == 'S과') | (test['등급명'] == 0)]   # 모든 상품에 대해서 수행하지 않고 GRAD_NM이 '상품', 'S과' 만 해당하는 품목 가져옴
            sep = sep[['datadate', '등급명', '조사단위(kg)', '가격(원)']]

            sep.rename(columns={"가격(원)": "가격"}, inplace=True)
            sep2 = sqldf(
                f"select datadate, max(가격) as '일자별_{text}가격_최대(원)', avg(가격) as '일자별_{text}가격_평균(원)', min(가격) as '일자별_{text}가격_최소(원)' from sep group by datadate")

            sep2.fillna(0,inplace=True)
#             if len(sep2) == 0:
#                 if name=='test':
#                     sep2 =  globals()['sep_'+test_dir+f'_{name}_{text}_0'] 
#                     sep2

#                 else:
#                     sep2 =  globals()[f'{name}_{text}_0'] 
#                     sep2[sep2.columns] = 0 
                
                
            if name=='test':
                globals()['sep_'+test_dir+f'_{name}_{text}_{k}'] = sep2
                
            else:
                globals()[f'{name}_{text}_{k}'] = sep2
 
                


        
                               
               
             

In [7]:
def pummok(주소,name='train'):
    if name == 'test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):

        try:
            pummok = pd.read_csv(주소+f'/pummok_{i}.csv')
            if len(pummok) == 0 :
                pummok = pd.read_csv(주소+f'/pummok_0.csv')
                pummok = pummok.groupby('datadate').mean()
                pummok = pummok[['단가(원)','거래량','해당일자_전체평균가격(원)']]
                pummok[pummok.columns] = 0
                pummok = pummok.reset_index()
                
            else:   
                pummok = pummok.groupby('datadate').mean()
                pummok = pummok[['단가(원)','거래량','해당일자_전체평균가격(원)']]
                pummok = pummok.reset_index()
                pummok.fillna(0,inplace = True)

        except:
            continue

        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_pummok_{i}'] = pummok

        else:
            globals()[f'{name}_pummok_{i}'] = pummok


In [8]:
def imexport(주소, name='train'):
    if name == 'test':
        test_dir = 주소.split('/')[-2][-1]
    for i in tqdm(range(0,37)):
        try:
            imexport=pd.read_csv(주소+f'/imexport_{i}.csv')
            
            if len(imexport) == 0:
                imexport = pd.read_csv(주소+f'/imexport_0.csv')
                imexport = imexport.groupby('datadate').mean()
                imexport[imexport.columns] = 0

            else:  
                imexport = imexport.groupby('datadate').mean()


        except:
            
            imexport = pd.read_csv(주소+f'/imexport_0.csv')
            imexport = imexport.groupby('datadate').mean()
            imexport[imexport.columns] = 0
            pass


        imexport.fillna(0,inplace=True)
        if name=='test':
            globals()['sep_'+test_dir+f'_{name}_imexport_{i}'] = imexport

        else:
            globals()[f'{name}_imexport_{i}'] = imexport


    
    

In [9]:
def make_csv(주소,name = 'train'):
    imexport(주소,name)
    pummok(주소,name)
    add_dosomae(주소,name)
    add_dosomae(주소,name,2)
    weather(주소,name)
    for i in range(0,37):
        if name == 'test':

            num = 주소.split('/')[-2][-1]
            
            
            temp = globals()[f'sep_{num}_{name}_pummok_{i}']

            temp1 = globals()[f'sep_{num}_{name}_weather_{i}']

            temp2 = globals()[f'sep_{num}_{name}_domae_{i}']
   
            temp3 = globals()[f'sep_{num}_{name}_somae_{i}']

            temp4 = globals()[f'sep_{num}_{name}_imexport_{i}']
            temp.set_index('datadate',inplace=True)
            temp1.set_index('datadate',inplace=True)
            temp2.set_index('datadate',inplace=True)
            temp3.set_index('datadate',inplace=True)

            temp = temp.join(temp1)
            temp = temp.join(temp2)
            temp = temp.join(temp3)
            temp.reset_index(inplace = True)
            temp['datadate'] = temp['datadate'].astype('str')
            temp['yyyymm'] =  temp['datadate'].apply(lambda x: x[:6] )
            temp4.reset_index(inplace=True)
            temp4['yyyymm'] = temp4['datadate'].astype('str')
            temp4.drop('datadate',inplace=True,axis=1)
            temp = temp.merge(temp4, how='left',on='yyyymm')
            temp.set_index('datadate',inplace = True)
            temp.drop('yyyymm',inplace=True,axis=1)
            globals()[f'{name}_total_sep_{num}_{i}'] = temp


        else:
            temp = globals()[f'{name}_pummok_{i}']
            temp1 = globals()[f'{name}_weather_{i}']
            temp2 = globals()[f'{name}_domae_{i}']
            temp3 = globals()[f'{name}_somae_{i}']
            temp4 = globals()[f'{name}_imexport_{i}']
            temp.set_index('datadate',inplace=True)
            temp1.set_index('datadate',inplace=True)
            temp2.set_index('datadate',inplace=True)
            temp3.set_index('datadate',inplace=True)
            temp = temp.join(temp1)
            temp = temp.join(temp2)
            temp = temp.join(temp3)
            temp.reset_index(inplace = True)
            temp['datadate'] = temp['datadate'].astype('str')
            temp['yyyymm'] =  temp['datadate'].apply(lambda x: x[:6] )
            temp4.reset_index(inplace=True)
            temp4['yyyymm'] = temp4['datadate'].astype('str')
            temp4.drop('datadate',inplace=True,axis=1)
            temp = temp.merge(temp4, how='left',on='yyyymm')
            temp.set_index('datadate',inplace = True)
            temp.drop('yyyymm',inplace=True,axis=1)
            globals()[f'{name}_total_{i}'] = temp
        
    

In [10]:
# train_data만들기 train_total_0 ~ train_total_36까지 만들어준다.
make_csv('/content/drive/MyDrive/농산물예측/aT_data/aT_train_raw/','train')

100%|██████████| 37/37 [00:16<00:00,  2.27it/s]


In [11]:
# 품목별로 컬럼 확인
for i in range(37):
  print(f"{i} 품목의 selected_columns : {list(globals()[f'train_total_{i}'].columns)}")

0 품목의 selected_columns : ['단가(원)', '거래량', '해당일자_전체평균가격(원)', '초기온도(℃)_0', '최대온도(℃)_0', '최저온도(℃)_0', '평균온도(℃)_0', '강수량(ml)_0', '습도(%)_0', '초기온도(℃)_1', '최대온도(℃)_1', '최저온도(℃)_1', '평균온도(℃)_1', '강수량(ml)_1', '습도(%)_1', '초기온도(℃)_2', '최대온도(℃)_2', '최저온도(℃)_2', '평균온도(℃)_2', '강수량(ml)_2', '습도(%)_2', '일자별_domae가격_최대(원)', '일자별_domae가격_평균(원)', '일자별_domae가격_최소(원)', '일자별_somae가격_최대(원)', '일자별_somae가격_평균(원)', '일자별_somae가격_최소(원)', '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)']
1 품목의 selected_columns : ['단가(원)', '거래량', '해당일자_전체평균가격(원)', '초기온도(℃)_0', '최대온도(℃)_0', '최저온도(℃)_0', '평균온도(℃)_0', '강수량(ml)_0', '습도(%)_0', '초기온도(℃)_1', '최대온도(℃)_1', '최저온도(℃)_1', '평균온도(℃)_1', '강수량(ml)_1', '습도(%)_1', '초기온도(℃)_2', '최대온도(℃)_2', '최저온도(℃)_2', '평균온도(℃)_2', '강수량(ml)_2', '습도(%)_2', '일자별_domae가격_최대(원)', '일자별_domae가격_평균(원)', '일자별_domae가격_최소(원)', '일자별_somae가격_최대(원)', '일자별_somae가격_평균(원)', '일자별_somae가격_최소(원)', '수출중량(kg)', '수출금액(달러)', '수입중량(kg)', '수입금액(달러)', '무역수지(달러)']
2 품목의 selected_columns : ['단가(원)', '거래량', '해당일자_전체평균

In [12]:
def scaling_df(df):
  scaler = StandardScaler()
  
  
  tmp = df.drop('해당일자_전체평균가격(원)',axis=1).copy()
  scaler.fit(tmp)
  df_scaled = scaler.transform(tmp)
  df_scaled = pd.DataFrame(data=df_scaled, columns = tmp.columns)
  df_scaled['log_target'] = np.log1p(df['해당일자_전체평균가격(원)']).values 
  return df_scaled

In [13]:
for i in range(37):
  globals()[f'train_scaled_{i}'] = scaling_df(globals()[f'train_total_{i}'])

In [14]:
train_scaled_0

,단가(원),거래량,초기온도(℃)_0,최대온도(℃)_0,최저온도(℃)_0,평균온도(℃)_0,강수량(ml)_0,습도(%)_0,초기온도(℃)_1,최대온도(℃)_1,...,일자별_domae가격_최소(원),일자별_somae가격_최대(원),일자별_somae가격_평균(원),일자별_somae가격_최소(원),수출중량(kg),수출금액(달러),수입중량(kg),수입금액(달러),무역수지(달러),log_target
0,-1.562158,-0.712850,-0.982587,-1.081306,-0.968576,-0.999390,-0.338411,0.0,-0.955433,-1.124811,...,-1.350599,-1.324653,-1.342485,-1.342761,-0.164255,-0.172015,2.401996,1.343234,-1.346349,0.000000
1,6.636992,-0.642555,-0.366690,-0.871961,-0.400946,-0.553287,-0.338411,0.0,-0.907740,-0.961424,...,0.713407,0.577939,0.567900,0.777304,-0.164255,-0.172015,2.401996,1.343234,-1.346349,9.938528
2,0.611502,-0.241407,-0.734390,-1.026694,-0.848170,-0.981182,-0.338411,0.0,-0.316343,-1.047923,...,0.713407,0.577939,0.567900,0.777304,-0.164255,-0.172015,2.401996,1.343234,-1.346349,8.132373
3,1.017981,-0.267114,-0.780352,-0.862859,-0.693362,-0.789995,-0.338411,0.0,-1.050819,-0.894147,...,0.713407,0.577939,0.567900,0.777304,-0.164255,-0.172015,2.401996,1.343234,-1.346349,8.787969
4,1.516286,-0.017604,-1.046934,-0.917471,-0.830969,-0.935661,-0.338411,0.0,-1.136667,-0.894147,...,0.713407,0.577939,0.567900,0.777304,-0.164255,-0.172015,2.401996,1.343234,-1.346349,8.366389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,-1.562158,-0.712850,-1.322708,-0.844655,-1.002978,-0.708057,-0.338411,0.0,-1.050819,-0.903758,...,-1.350599,-1.324653,-1.342485,-1.342761,-0.164255,-0.172015,1.133965,1.199612,-1.203526,0.000000
1457,1.422662,0.186516,-0.477000,-0.626207,-0.590156,-0.562391,-0.338411,0.0,-0.898201,-0.701927,...,1.140443,1.532417,1.025181,0.856559,-0.164255,-0.172015,1.133965,1.199612,-1.203526,8.720925
1458,1.433806,-0.363770,-0.918239,-1.572813,-0.959976,-1.226994,-0.338411,0.0,-0.964971,-1.144033,...,1.140443,1.532417,1.021672,0.856559,-0.164255,-0.172015,1.133965,1.199612,-1.203526,8.801625
1459,1.881196,-0.091696,-1.322708,-2.173544,-1.648012,-2.055473,-0.338411,0.0,-0.984049,-1.951357,...,1.140443,1.532417,1.021672,0.856559,-0.164255,-0.172015,1.133965,1.199612,-1.203526,8.790799


In [15]:
try_cnt = 6 # 모델이나 체크포인트를 각자 다른 폴더에 저장해주기 위해
            # 새로운거 시도할때마다 숫자 바꿔서 내면된다.

In [16]:
def make_Tensor(array):
    return tf.convert_to_tensor(array, dtype=tf.float32)

def astype_data(data):
    df = data.astype(np.float32)
    return make_Tensor(df)

In [17]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):

    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [18]:
def build_model(input_shape, head_size, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0, mlp_dropout=0):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(28)(x) # 4주 예측
    return keras.Model(inputs, outputs)

In [19]:
def call_back_set(name, epoch, batch_size):
    early_stopping = EarlyStopping(monitor='val_loss', patience=100)

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}') == False:
        os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}')

    filename = f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/{name}-{epoch}-{batch_size}.h5'

    checkpoint = ModelCheckpoint(filename,
                                 monitor='val_loss',
                                 verbose=1,
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='auto'
                                 )
    return [early_stopping, checkpoint]

In [20]:
def train(x_train, y_train, x_val, y_val, name, epoch, batch_size, learning_rate = 0.001, verbose = 1):


    model = build_model(
    x_train.shape[1:],
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
    )

    model.compile(
        loss="mean_squared_error",
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate)
    )


    # Train the model
    with tf.device('/device:GPU:0'):
        history1 = model.fit(
            x_train, y_train,
            epochs = epoch,
            steps_per_epoch=len(x_train) / batch_size,
            batch_size=batch_size,
            validation_data=(x_val, y_val),
            validation_steps=len(x_val) / batch_size,
            shuffle=False,
            callbacks=call_back_set(name, epoch, batch_size),
            verbose=verbose)

    return model

In [21]:
def time_window(df, t, t_sep):
    seq_len = t
    seqence_length = seq_len + t_sep

    result = []
    for index in tqdm(range(len(df) - seqence_length)):
        result.append(df[index: index + seqence_length].values)

    return np.array(result)

In [22]:

# data_list = glob('/content/drive/MyDrive/농산물예측/aT_data/data/train/*.csv')
epoch = 1000
batch = 15
# tr_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 '] # train 에서 사용하지 않는 열
# ts_del_list = ['단가(원)', '거래량', '거래대금(원)', '경매건수', '도매시장코드', '도매법인코드', '산지코드 ', '해당일자_전체평균가격(원)'] # test 에서 사용하지 않는 열
#check_col = ['일자구분_중순', '일자구분_초순', '일자구분_하순','월구분_10월', '월구분_11월', '월구분_12월', '월구분_1월', '월구분_2월', '월구분_3월', 
#             '월구분_4월','월구분_5월', '월구분_6월', '월구분_7월', '월구분_8월', '월구분_9월'] # 열 개수 맞추기

In [23]:
for i in range(37):


    df_number = i
    df = globals()[f'train_scaled_{i}']
    
    # nan 처리
    df.fillna(0, inplace = True)
    
  

    

    # 변수와 타겟 분리
    x, y = df[[i for i in df.columns if i != 'log_target']], df['log_target']

    # 2주 입력을 통한 이후 4주 예측을 위해 y의 첫 14일을 제외
    y = y[14:]

    # time series window 생성
    data_x = time_window(x, 13, 1)
    data_y = time_window(y, 27, 1)

    # y의 길이와 같은 길이로 설정
    xdata = data_x[:len(data_y)]
    ydata = data_y

    # train, validation 분리 (8 : 2)
    x_train, x_val, y_train, y_val = train_test_split(xdata, ydata, test_size=0.2, shuffle=False, random_state=119)

    # transformer 모델 훈련
    transformer = train(astype_data(x_train), y_train, astype_data(x_val), y_val, f'transformer-{df_number}', epoch, batch)
    transformer.load_weights(f'/content/drive/MyDrive/농산물예측/aT_data/check{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}') == False:
         os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}')

    
    
    
    # 모델 저장
    transformer.save(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{df_number}-{epoch}-{batch}.h5')

100%|██████████| 1419/1419 [00:00<00:00, 35484.97it/s]


Epoch 1/1000
76/75 [==============================] - ETA: 0s - loss: 26.7219
Epoch 1: val_loss improved from inf to 7.94448, saving model to /content/drive/MyDrive/농산물예측/aT_data/check6/transformer-35-1000-15.h5
75/75 [==============================] - 15s 18ms/step - loss: 26.7219 - val_loss: 7.9445
Epoch 2/1000
76/75 [==============================] - ETA: 0s - loss: 12.1333
Epoch 2: val_loss did not improve from 7.94448
75/75 [==============================] - 1s 11ms/step - loss: 12.1333 - val_loss: 10.0033
Epoch 3/1000
72/75 [===========================>..] - ETA: 0s - loss: 13.0421
Epoch 3: val_loss did not improve from 7.94448
75/75 [==============================] - 1s 11ms/step - loss: 13.2267 - val_loss: 19.9079
Epoch 4/1000
76/75 [==============================] - ETA: 0s - loss: 13.5305
Epoch 4: val_loss did not improve from 7.94448
75/75 [==============================] - 1s 11ms/step - loss: 13.5305 - val_loss: 8.0260
Epoch 5/1000
75/75 [============================>.] - 

100%|██████████| 1419/1419 [00:00<00:00, 33827.92it/s]


Epoch 1/1000
76/75 [==============================] - ETA: 0s - loss: 23.3820
Epoch 1: val_loss improved from inf to 12.98881, saving model to /content/drive/MyDrive/농산물예측/aT_data/check6/transformer-36-1000-15.h5
75/75 [==============================] - 4s 18ms/step - loss: 23.3820 - val_loss: 12.9888
Epoch 2/1000
75/75 [============================>.] - ETA: 0s - loss: 13.5145
Epoch 2: val_loss improved from 12.98881 to 12.31643, saving model to /content/drive/MyDrive/농산물예측/aT_data/check6/transformer-36-1000-15.h5
75/75 [==============================] - 1s 13ms/step - loss: 13.4737 - val_loss: 12.3164
Epoch 3/1000
76/75 [==============================] - ETA: 0s - loss: 11.1168
Epoch 3: val_loss improved from 12.31643 to 7.22077, saving model to /content/drive/MyDrive/농산물예측/aT_data/check6/transformer-36-1000-15.h5
75/75 [==============================] - 1s 13ms/step - loss: 11.1168 - val_loss: 7.2208
Epoch 4/1000
76/75 [==============================] - ETA: 0s - loss: 9.2294
Epoch 

In [24]:
# from tensorflow.python.ops.gen_data_flow_ops import tensor_array_concat_v3_eager_fallback
for i in tqdm(range(10)):
  
  make_csv(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{i}/','test') 
  
  for j in range(37):
    # print(f'sep:{i}, 품목:{j}')
    # print(len(globals()[f'test_total_sep_{i}_{j}'])) # df row수 - 0이 있나 확인하기 위해서

    # 사용할 열 선택 - 정규식 적용후, train 데이터랑 똑같은 컬럼만 남기기

    # 정규 표현식 적용
    #globals()[f'test_total_sep_{i}_{j}'].columns = globals()[f'test_total_sep_{i}_{j}'].columns.str.replace(r'\([^)]*\)','',regex= True)

    # 스케일링 & 타겟값 변환
    #globals()[f'train_scaled_{i}'] = scaling_df(globals()[f'total_{i}'])
    globals()[f'test_total_sep_{i}_{j}'] = scaling_df(globals()[f'test_total_sep_{i}_{j}'])

    # train 데이터랑 똑같이 selection
    globals()[f'test_total_sep_{i}_{j}'] = globals()[f'test_total_sep_{i}_{j}'][list(globals()[f'train_scaled_{j}'].columns)]

    

    # 해당일자평균가격 테스트 데이터에서는 타겟값 없애고 돌리네
    globals()[f'test_total_sep_{i}_{j}'].drop('log_target',axis=1, inplace=True)


    file_number = j

    # nan 처리
    globals()[f'test_total_sep_{i}_{j}'].fillna(0, inplace = True)
    # 형상 맞추기 코드는 아직 넣지 않았다. (안넣어도 되지 않을까?)

    # x_test 생성
    df_test = astype_data(globals()[f'test_total_sep_{i}_{j}'].values.reshape(1, globals()[f'test_total_sep_{i}_{j}'].values.shape[0], globals()[f'test_total_sep_{i}_{j}'].values.shape[1]))



    # model test
    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}')

    if os.path.exists(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}') == False:
            os.mkdir(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}')

    # 해당하는 모델 불러오기
    model_test = tf.keras.models.load_model(f'/content/drive/MyDrive/농산물예측/aT_data/model{try_cnt}/transformer-{file_number}-{epoch}-{batch}.h5')
    pred = model_test.predict(df_test)


    # 여기서 다시 형변환 해주자!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!np.expm1
    pred = np.expm1(pred)




    # 결과 저장
    save_df = pd.DataFrame(pred).T
    save_df.to_csv(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{i}/predict_{file_number}.csv', index=False)


100%|██████████| 37/37 [00:11<00:00,  3.23it/s]

100%|██████████| 37/37 [00:11<00:00,  3.23it/s]

100%|██████████| 37/37 [00:03<00:00, 11.20it/s]

100%|██████████| 37/37 [00:01<00:00, 18.54it/s]

100%|██████████| 37/37 [00:10<00:00,  3.37it/s]

100%|██████████| 37/37 [00:10<00:00,  3.41it/s]

100%|██████████| 37/37 [00:03<00:00, 11.16it/s]

100%|██████████| 37/37 [00:02<00:00, 18.50it/s]

100%|██████████| 37/37 [00:10<00:00,  3.58it/s]

100%|██████████| 37/37 [00:11<00:00,  3.33it/s]

100%|██████████| 37/37 [00:03<00:00, 11.90it/s]

100%|██████████| 37/37 [00:02<00:00, 18.00it/s]

100%|██████████| 37/37 [00:10<00:00,  3.58it/s]

100%|██████████| 37/37 [00:11<00:00,  3.21it/s]

100%|██████████| 37/37 [00:03<00:00, 11.82it/s]

100%|██████████| 37/37 [00:02<00:00, 17.82it/s]

100%|██████████| 37/37 [00:10<00:00,  3.63it/s]

100%|██████████| 37/37 [00:12<00:00,  2.98it/s]

100%|██████████| 37/37 [00:03<00:00,  9.75it/s]

100%|██████████| 37/37 [00:02<00:00, 18.04it/s]

100%|██████████| 37/

In [25]:
# 이부분 세부적으로 다 바꿔야한다.



for k in tqdm(range(10)):

  globals()[f'set_df_{k}'] = pd.DataFrame()
  answer_df_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/model_output{try_cnt}/set_{k}/*.csv') # 예측한 결과 불러오기
  pum_list = glob(f'/content/drive/MyDrive/농산물예측/aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
  pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

  for i in answer_df_list:
    df = pd.read_csv(i)
    number = i.split('_')[-1].split('.')[0]

    base_number = 0
    for p in pummok:
      if number == p.split('_')[-1].split('.')[0]:
        pum_df = pd.read_csv(p)

        if len(pum_df) != 0:
           base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
        else:
          base_number = np.nan

    globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경

  globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

100%|██████████| 10/10 [00:03<00:00,  2.91it/s]


In [26]:
set_df_0

,품목0,품목1,품목2,품목3,품목4,품목5,품목6,품목7,품목8,품목9,...,품목27,품목28,품목29,품목30,품목31,품목32,품목33,품목34,품목35,품목36
0,3871.125000,1362.117613,2909.783785,3400.075583,3947.809169,9253.947514,2717.280000,3361.030923,4911.899864,1173.018633,...,8640.811309,602.005658,1105.412623,1566.274239,3633.464557,5454.710444,5619.188362,5230.620027,2905.100888,2087.675036
1,680.322270,479.136700,1944.933000,2400.777000,459.995820,1479.317100,67.622960,1113.092800,1853.967300,243.661830,...,13369.688000,158.145890,1071.290600,262.190430,592.333440,354.353360,7497.166500,746.658450,1329.495800,435.482060
2,24379.518000,774.891100,731.963560,894.149200,1990.323500,8058.347700,593.837650,2704.948500,5703.565400,2914.620800,...,1813.357400,414.498100,372.189450,590.163700,3819.718300,5848.906700,3361.145500,3345.885700,1497.609600,1113.033800
3,0.564044,3.970380,4.818207,2.754635,0.272738,1.505403,1494.826700,5.699214,8.380516,5.218194,...,4.475069,0.259369,2.519033,1.304400,4.460565,1.926234,-0.093244,1.298456,0.144410,3.255756
4,15872.952000,985.231700,3198.028300,3120.268600,2238.227500,7705.251500,658.149660,14153.055000,2500.327400,1894.441200,...,12606.523000,516.446500,2910.101600,1403.330300,4254.203600,6388.182000,9487.683000,4161.900000,2859.670400,1891.848000
5,10251.464000,941.900630,5911.229000,3082.604500,3012.698700,7811.311500,502.274440,12410.174000,6437.968000,2480.920000,...,22262.186000,716.822270,2622.051800,1243.263300,4594.087400,4542.004400,10838.930000,5838.873000,3124.072000,2049.926000
6,8123.130400,1078.471300,3914.713100,3171.239000,991.715900,7538.979000,460.989170,16483.385000,4687.954000,1863.533200,...,26313.904000,504.732540,2907.218800,1187.768000,4972.309600,4275.370000,12717.687500,4426.274000,2741.465600,1725.430300
7,15483.377000,1065.867200,7377.490700,3437.097200,2324.452400,9255.068000,1063.431200,15304.204000,8440.083000,2494.747000,...,21841.200000,634.064000,2868.372800,1374.687500,5277.976000,3947.958300,13226.145500,4631.041500,3042.952600,1709.441200
8,811.113650,418.814540,1548.981400,2369.822800,313.711550,1448.518800,60.172020,1265.907700,1414.356200,206.137250,...,9619.402000,187.429470,803.308500,186.692260,584.483200,289.826720,10154.502000,734.806640,1545.324500,499.385250
9,24713.812000,669.767330,708.212100,874.539550,1499.058800,7493.849600,504.446140,2732.871600,5687.919000,2517.614700,...,1314.161300,455.827450,343.750060,493.524440,3677.338900,5277.790000,4428.415000,3022.613300,1638.654400,1260.497400


In [27]:
date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']


for k in range(10):
  globals()[f'answer_df_{k}'] = pd.DataFrame()
  for c in globals()[f'set_df_{k}'].columns:
    base_d = globals()[f'set_df_{k}'][c][0] # 변동률 기준 t 값

    ans_1_14 = []
    for i in range(14):
      ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

    ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

    globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
  globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
  globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

In [28]:
# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
  if i== 0 :
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
  else:
    all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])


all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

In [29]:
# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('/content/drive/MyDrive/농산물예측/aT_data/answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'/content/drive/MyDrive/농산물예측/submit{try_cnt}.csv',index=False)

In [30]:
all_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.824257,-0.648241,-0.331588,-0.293905,-0.883481,-0.840142,-0.975114,-0.668824,...,0.547272,-0.737302,-0.030868,-0.832602,-0.836978,-0.935037,0.334208,-0.857252,-0.542358,-0.791403
1,0,d+2,5.297786,-0.431113,-0.748447,-0.737021,-0.495841,-0.129199,-0.781459,-0.195203,...,-0.790140,-0.311471,-0.663303,-0.623205,0.051261,0.072267,-0.401845,-0.360327,-0.484490,-0.466855
2,0,d+3,-0.999854,-0.997085,-0.998344,-0.999190,-0.999931,-0.999837,-0.449881,-0.998304,...,-0.999482,-0.999569,-0.997721,-0.999167,-0.998772,-0.999647,-1.000017,-0.999752,-0.999950,-0.998440
3,0,d+4,3.100346,-0.276691,0.099060,-0.082294,-0.433046,-0.167355,-0.757791,3.210927,...,0.458951,-0.142124,1.632593,-0.104033,0.170839,0.171131,0.688444,-0.204320,-0.015638,-0.093801
4,0,d+5,1.648187,-0.308503,1.031501,-0.093372,-0.236868,-0.155894,-0.815155,2.692371,...,1.576400,0.190723,1.372012,-0.206229,0.264382,-0.167324,0.928914,0.116287,0.075375,-0.018082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.015527,-0.447839,1.036406,-0.248063,-0.780943,0.300467,-0.099713,NaN,...,0.014345,0.208980,-0.335579,0.316426,-0.096606,-0.269083,1.452067,0.282786,0.010517,-0.343582
146,9,d+12,0.201655,-0.329604,0.784412,-0.283628,-0.189728,0.669573,-0.875652,NaN,...,0.382740,0.609632,-0.255838,0.348259,-0.085377,-0.478466,2.174774,0.076882,-0.246650,-0.055175
147,9,d+13,0.074489,-0.498237,0.034500,-0.291913,-0.737900,0.399112,-0.033642,NaN,...,-0.427893,0.164677,-0.299720,-0.136043,-0.085839,-0.568664,2.874284,-0.033945,-0.030667,-0.278213
148,9,d+14,0.186825,-0.392794,-0.105676,-0.535549,-0.542387,0.847652,-0.950279,NaN,...,0.060414,0.386318,-0.316200,-0.049934,-0.073461,-0.454122,3.674288,0.420391,-0.275357,-0.289245


In [31]:
out_ans

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,d+2,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,d+5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146,9,d+12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,9,d+13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
148,9,d+14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
submit_df

,Set,일자,품목0 변동률,품목1 변동률,품목2 변동률,품목3 변동률,품목4 변동률,품목5 변동률,품목6 변동률,품목7 변동률,...,품목27 변동률,품목28 변동률,품목29 변동률,품목30 변동률,품목31 변동률,품목32 변동률,품목33 변동률,품목34 변동률,품목35 변동률,품목36 변동률
0,0,d+1,-0.824257,-0.648241,-0.331588,-0.293905,-0.883481,-0.840142,-0.975114,-0.668824,...,0.547272,-0.737302,-0.030868,-0.832602,-0.836978,-0.935037,0.334208,-0.857252,-0.542358,-0.791403
1,0,d+2,5.297786,-0.431113,-0.748447,-0.737021,-0.495841,-0.129199,NaN,-0.195203,...,-0.790140,-0.311471,-0.663303,-0.623205,0.051261,0.072267,-0.401845,-0.360327,-0.484490,-0.466855
2,0,d+3,NaN,NaN,NaN,NaN,NaN,NaN,-0.449881,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,d+4,3.100346,-0.276691,0.099060,-0.082294,-0.433046,-0.167355,NaN,3.210927,...,0.458951,-0.142124,1.632593,-0.104033,0.170839,0.171131,0.688444,-0.204320,-0.015638,-0.093801
4,0,d+5,1.648187,-0.308503,1.031501,-0.093372,-0.236868,-0.155894,NaN,2.692371,...,1.576400,0.190723,1.372012,-0.206229,0.264382,-0.167324,0.928914,0.116287,0.075375,-0.018082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,9,d+11,0.015527,-0.447839,1.036406,-0.248063,-0.780943,0.300467,-0.099713,NaN,...,0.014345,0.208980,-0.335579,0.316426,-0.096606,-0.269083,1.452067,0.282786,0.010517,-0.343582
146,9,d+12,0.201655,-0.329604,0.784412,-0.283628,-0.189728,0.669573,-0.875652,NaN,...,0.382740,0.609632,-0.255838,0.348259,-0.085377,-0.478466,2.174774,0.076882,-0.246650,-0.055175
147,9,d+13,0.074489,-0.498237,0.034500,-0.291913,-0.737900,0.399112,-0.033642,NaN,...,-0.427893,0.164677,-0.299720,-0.136043,-0.085839,-0.568664,2.874284,-0.033945,-0.030667,-0.278213
148,9,d+14,0.186825,-0.392794,-0.105676,-0.535549,-0.542387,0.847652,-0.950279,NaN,...,0.060414,0.386318,-0.316200,-0.049934,-0.073461,-0.454122,3.674288,0.420391,-0.275357,-0.289245
